## 2.0 Data Pre - Processing
- **Import libraries:**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

- **Show higher amount of columns and rows in a pandas data frame:**

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5000)

- **Import in a pandas data frame the csv cleaned and featured file:**

In [3]:
df = pd.read_csv("C:\\Users\\aless\\Desktop\\JATO\\Dataframe\\Clean\\df_vehicles.csv")

#### Now i have to convert from categorical to numerical the following columns in order to render compatible each columns to the K Means model:
- **segment**;
- **trim**;
- **transmission type**;
- **version**;
- **driven wheel**;
- **bodystyle**;
- **fuel**;
- **model**;
- **make**;

Being binary the following column **'transmission type'**, I will transform to boolean 1 \ 0:

In [4]:
df['transmission type'] = df['transmission type'].map({'M': 1, 'A': 0})

Now I want to pre process the following columns: **'driven wheel'**, **'fuel'** and **'bodystyle'** which contains three unique values for the first and five for the second one using one hot encoder:

In [5]:
# Now I instantiate the encoder in a variable:
encoder_1 = OneHotEncoder(sparse=False)
encoder_2 = OneHotEncoder(sparse=False)
encoder_3 = OneHotEncoder(sparse=False)

# I do encoding for the first and the second column:
encoded_array_1 = encoder_1.fit_transform(df[['driven wheel']])
encoded_array_2 = encoder_2.fit_transform(df[['fuel']])
encoded_array_3 = encoder_3.fit_transform(df[['bodystyle']])

# Get the feature names:
feature_names_1 = encoder_1.get_feature_names(['driven wheel'])
feature_names_2 = encoder_2.get_feature_names(['fuel'])
feature_names_3 = encoder_3.get_feature_names(['bodystyle'])
                                              
# Create a DataFrame with the new columns:
encoded_df_1 = pd.DataFrame(encoded_array_1, columns=feature_names_1)
encoded_df_2 = pd.DataFrame(encoded_array_2, columns=feature_names_2)
encoded_df_3 = pd.DataFrame(encoded_array_3, columns=feature_names_3)
                                              
# Concatenate the original DataFrame and the new one:
df = pd.concat([df, encoded_df_1], axis=1)
df = pd.concat([df, encoded_df_2], axis=1)
df = pd.concat([df, encoded_df_3], axis=1)

# Finally I Drop the original 'driven wheel' and 'fuel' columns:
df.drop('driven wheel', axis=1, inplace=True)
df.drop('fuel', axis=1, inplace=True)
df.drop('bodystyle', axis=1, inplace=True)

Now I will do pre processing among the columns named **'make'** and **'model'** using LabelEncoder:

In [6]:
# Instantiate a new variable containing the label encoder model:
label_encoder_1 = LabelEncoder()
label_encoder_2 = LabelEncoder()
label_encoder_3 = LabelEncoder()

# I now Apply the label encoder model to the column named make and model:
df['make_encoded'] = label_encoder_1.fit_transform(df['make'])
df['model_encoded'] = label_encoder_2.fit_transform(df['model'])
df['segment_encoded'] = label_encoder_3.fit_transform(df['segment'])

# Now I will drop the columns original that i do not need anymore:
df.drop('make', axis=1, inplace=True)
df.drop('model', axis=1, inplace=True)
df.drop('segment', axis=1, inplace=True)

Now I will do the pre processing phase among the column named  as **'trim'**, adopting the method 

- For the pre-processing phase applied to this column, I follow the technique mentioned in Niculescu-Mizil et al. (2009). - - By selecting the top N categories based on value counts in the categorical variable, I create N dummy variables for each item, indicating whether it belongs to the top N categories by number.
- Thus, in the end, I will have ten additional columns, one for each of the most frequent categories (Top N), without having hundreds or thousands of extra dummy columns that would compromise the dimensionality and usability of the data frame;
- In this case I decided to take the first 30 values per counts since these counts at least 1% of the df after cheked with the .value_counts(1) method showing the related % of counts.

In [7]:
top_30 = [x for x in df["trim"].value_counts().sort_values(ascending=False).head(30).index]
def one_hot_top_x(df, variable, top_x_labels):
    for label in top_x_labels:
        df[variable + "_" + label] = np.where(df[variable] == label, 1, 0)

# here I apply the UDF just created using the trim column as parameter:
one_hot_top_x(df, "trim", top_30)

# Now I will drop the column original that i do not need anymore:
df.drop('trim', axis=1, inplace=True)

Now the last object column remained, the **'version'** column. Since this has more than 6000 unique values I inted using the frequency encoding method:

In [8]:
# I create a list of values per each unique values in my df version column
# where i have specifically the frequency terms per each of the unique values:
version_freq_encoding = df['version'].value_counts() / len(df)

# Using hte same list I now map the new column ancoded
df['version_frequency'] = df['version'].map(version_freq_encoding)

# As last I now drop the original not encoded column version:
df.drop(columns=['version'], inplace=True)

Drop the column regarding the vehicle index which is not necessary for the clusterization of the cars:

In [9]:
df.drop('vehicle_id', axis=1, inplace=True)

Reset the final index:

In [10]:
df.reset_index(drop=True, inplace=True)

Check of the final pre processed data frame:

In [11]:
df.shape

(66560, 86)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66560 entries, 0 to 66559
Data columns (total 86 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   power (hp)                66560 non-null  float64
 1   number doors              66560 non-null  float64
 2   engine ltr                66560 non-null  float64
 3   transmission type         66560 non-null  int64  
 4   length                    66560 non-null  float64
 5   width                     66560 non-null  float64
 6   height                    66560 non-null  float64
 7   price                     66560 non-null  float64
 8   year                      66560 non-null  float64
 9   MVLJAN                    66560 non-null  float64
 10  MVLFEB                    66560 non-null  float64
 11  MVLMAR                    66560 non-null  float64
 12  MVLAPR                    66560 non-null  float64
 13  MVLMAY                    66560 non-null  float64
 14  MVLJUN

In [13]:
null_counts = df.isnull().sum()
total_rows = len(df)
null_percentages = (null_counts / total_rows) * 100
null_data = pd.DataFrame({'Column': null_counts.index,
                          'Null_Count': null_counts.values,
                          'Null_Percentage': null_percentages.values})
null_data

,Column,Null_Count,Null_Percentage
0,power (hp),0,0.0
1,number doors,0,0.0
2,engine ltr,0,0.0
3,transmission type,0,0.0
4,length,0,0.0
5,width,0,0.0
6,height,0,0.0
7,price,0,0.0
8,year,0,0.0
9,MVLJAN,0,0.0


Save the pre processed data frame to a new csv file ready for the clusterization:

In [14]:
df_csv = df.to_csv('C:\\Users\\aless\\Desktop\\JATO\\Dataframe\\Final\\df_pre_processed.csv', index=False)